## Imports

In [1]:
# Libraries

import re
import requests
import numpy as np
import pandas as pd
import sqlite3
import sqlalchemy

import logging
import os

from datetime import datetime

from bs4 import BeautifulSoup

from sqlalchemy import create_engine

# Data Colletion
## Job 01: Extraction

### 0.1. Loading data
def get_showroom_data(url, headers):
    page = requests.get( url, headers=headers )
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # 1.0. Scrape data - Showroom products
    products = soup.find('ul', class_ = 'products-listing small')
    product_list = products.find_all('article', class_= 'hm-product-item')
    
    # product id
    product_id = [p.get('data-articlecode') for p in product_list]
    
    # product type
    product_type = [p.get('data-category') for p in product_list]
    
    # product name
    product_list = products.find_all('a', class_='link')
    product_name = [p.get_text() for p in product_list]
    
    # product price
    product_list = products.find_all('span', class_='price regular')
    product_price = [p.get_text() for p in product_list]
    
    # dataFrame creation
    data = pd.DataFrame([product_id,
                         product_name,
                         product_type,
                         product_price,]).T
    
    # Rename colomns DataFrame
    data.columns = ['product_id',
                    'product_type',
                    'product_name',
                    'product_price']
    
    # scrapy datetime
    data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return data

# Data Collection by Product
## Job 02: Transformation

def get_product_details( data ):
    # 2.0. Scrape data - Products Details
    
    df_details = pd.DataFrame()
    
    # unique columns for all products
    aux = []
    
    set(aux)
    
    cols = ['Art. No.',
            'Care instructions',
            'Composition',
            'Concept',
            'Description',
            'Fit',
            'Imported',
            'Material',
            'Nice to know',
            'messages.garmentLength',
             'messages.waistRise']
    
    
    df_pattern = pd.DataFrame(columns = cols)
    
    # Looping for all products on the site
    for i in range(len(data)):
        url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] +'.html'
        print('Product: {}'.format(url))
        logger.debug('Product: %s',url)
        
        
        # Request
        page = requests.get( url, headers=headers )
        
        # Instantiating Beautiful Soup object
        soup = BeautifulSoup(page.text, 'html.parser')
    
        #============================ color_name =============================
        product_atributes_list = soup.find_all('a', {'class':['filter-option miniature', 'filter-option miniature active']})
        color_name = [p.get('data-color') for p in product_atributes_list]
    
        # product id
        product_id_c = [p.get('data-articlecode') for p in product_atributes_list]
    
        # DataFrame creation
        df_color = pd.DataFrame([product_id_c, color_name]).T
        
        # Rename colomns DataFrame
        df_color.columns = ['product_id', 'color_name']
        
        # ==================== Iterate over colors =================================
    
        for j in range( len( df_color ) ):
            url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j,'product_id'] + '.html'
            print( 'Color: {}'.format( url ) )
            logger.debug('Color: %s',url)
        
            page = requests.get( url, headers=headers )
        
            # Beautiful Soup object
            soup = BeautifulSoup( page.text, 'html.parser' )
    
        # Generate Style id + Color id
        df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
        df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])
    
    #============================ Composition =============================
    product_composition_list = soup.find_all('div', class_='details-attributes-list-item')
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]
    
    # rename DataFrame
    df_composition = pd.DataFrame(product_composition).T
    df_composition.columns = df_composition.iloc[0]
    
    # Delete First row
    df_composition = df_composition.iloc[1:].fillna(method='ffill')
    
    # garantee the same number of columns
    df_composition = pd.concat([df_pattern, df_composition], axis=0)
    
    # Generate Style id + Color id
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])
    
    aux = aux + df_composition.columns.tolist()
        
    # Merge Data Color + Data Composition
    data_color_composition = pd.merge(df_color, df_composition[['style_id','Fit','Composition', 'Material','Description','messages.waistRise']] , how='left', on='style_id')
    
    # All details products
    df_details = pd.concat([df_details, data_color_composition], axis=0)
        
    # Joining the DataFrames: data + details
    data['style_id'] = data['product_id'].apply(lambda x: x[:-3])
    data['color_id'] = data['product_id'].apply(lambda x: x[-3:])

    data = pd.merge(data, df_details[['style_id','color_name','Fit','Composition', 'Description','messages.waistRise']], how='left', on='style_id')
        
    return data

# Data Cleaning
## Job 03: Transformation

def data_cleaning( data ):
    # product id      
    data = data.dropna(subset=['product_id'])
    data['product_id'] = data['product_id'].astype( int )
    
    # product name
    data['product_name'] = data['product_name'].apply( lambda x: x.replace(' ', '_').lower())

    # product price
    data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '') )
    data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '') ).astype(float)
    
    # scrapy datetime
    data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format= '%Y-%m-%d %H:%M:%S')
    
    # color name
    data['color_name'] = data['color_name'].apply( lambda x: x.replace(' ', '_').replace('/', '_').lower() if pd.notnull(x) else x)

    # fit
    data['Fit'] = data['Fit'].apply( lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

    # composition
    data = data[~data['Composition'].str.contains( 'Pocket lining:', na=False )]
    #data = data[~data['Composition'].str.contains( 'Shell:', na=False )]
    
    # drop duplicates
    data = data.drop_duplicates( subset=['product_id', 'product_type','product_name', 'product_price','scrapy_datetime', 'color_id','color_name', 'Fit'], keep='last' )
    
    # reset index
    data = data.reset_index( drop=True )
    
    # break composition by comma
    df1 = data['Composition'].str.split( ',', expand=True )

    # ------ shell_cotton -------

    data['shell_cotton'] = ''
    data['shell_cotton'] = df1[0]
    data['shell_cotton'] = data['shell_cotton'].apply( lambda x: int(re.search ('\d+', x).group(0)) / 100 if pd.notnull(x) else x)


    # ------ shell_spandex -------

    data['shell_spandex'] = ''
    data['shell_spandex'] = df1[1]
    data['shell_spandex'] = data['shell_spandex'].apply( lambda x: int(re.search ('\d+', x).group(0)) / 100 if pd.notnull(x) else x)


    # Drop columns
    data = data.drop(columns=['Composition', 'Description'], axis=1)

    # Drop duplicates
    data = data.drop_duplicates()
    
    return data


    
    


Product: https://www2.hm.com/en_us/productpage.1074475001.html
Color: https://www2.hm.com/en_us/productpage.1074475001.html
Product: https://www2.hm.com/en_us/productpage.1071707001.html
Color: https://www2.hm.com/en_us/productpage.1071707001.html
Color: https://www2.hm.com/en_us/productpage.1071707002.html
Color: https://www2.hm.com/en_us/productpage.1071707008.html
Product: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256002.html
Color: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256004.html
Color: https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256006.html
Color: https://www2.hm.com/en_us/productpage.1024256007.html
Color: https://www2.hm.com/en_us/productpage.1024256008.html
Product: https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://w

Color: https://www2.hm.com/en_us/productpage.1004199002.html
Color: https://www2.hm.com/en_us/productpage.1004199003.html
Color: https://www2.hm.com/en_us/productpage.1004199004.html
Color: https://www2.hm.com/en_us/productpage.1004199005.html
Product: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256002.html
Color: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256004.html
Color: https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256006.html
Color: https://www2.hm.com/en_us/productpage.1024256007.html
Color: https://www2.hm.com/en_us/productpage.1024256008.html
Product: https://www2.hm.com/en_us/productpage.1004199002.html
Color: https://www2.hm.com/en_us/productpage.1004199001.html
Color: https://www2.hm.com/en_us/productpage.1004199002.html
Color: https://www2.

Color: https://www2.hm.com/en_us/productpage.0690449007.html
Color: https://www2.hm.com/en_us/productpage.0690449009.html
Color: https://www2.hm.com/en_us/productpage.0690449011.html
Color: https://www2.hm.com/en_us/productpage.0690449013.html
Color: https://www2.hm.com/en_us/productpage.0690449021.html
Color: https://www2.hm.com/en_us/productpage.0690449022.html
Color: https://www2.hm.com/en_us/productpage.0690449024.html
Color: https://www2.hm.com/en_us/productpage.0690449028.html
Color: https://www2.hm.com/en_us/productpage.0690449035.html
Color: https://www2.hm.com/en_us/productpage.0690449036.html
Color: https://www2.hm.com/en_us/productpage.0690449040.html
Color: https://www2.hm.com/en_us/productpage.0690449043.html
Color: https://www2.hm.com/en_us/productpage.0690449046.html
Color: https://www2.hm.com/en_us/productpage.0690449051.html
Color: https://www2.hm.com/en_us/productpage.0690449056.html
Product: https://www2.hm.com/en_us/productpage.0690449043.html
Color: https://www2.hm

In [4]:
data.shape

(38, 12)

## Database SQLite

In [2]:
data_insert = data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'color_name',
    'product_type',
    'Fit',
    'product_price',
    'messages.waistRise',
    'shell_cotton',
    'shell_spandex',
    'scrapy_datetime'   
]]

# Creat query
query_showroom_schema = """
    CREATE TABLE vitrine_one (
        product_id             INTEGER,
        style_id               INTEGER
        color_id               INTEGER
        product_name           TEXT,
        color_name             TEXT,
        product_type           TEXT,
        Fit                    TEXT,
        product_price          REAL,
        messages.waistRise     TEXT,
        shell_cotton           REAL,
        shell_spandex          REAL,
        scrapy_datetime        TEXT
        )
"""

try:
    # Connect to dataset
    conn = sqlite3.connect('data_base_hm.sqlite')
    cursor = conn.cursor()
    print("Database created and Successfully Connected to SQLite")
    
    
    query_showroom_schema = "select sqlite_version();"
    
    # Run
    cursor.execute(query_showroom_schema)
    conn.commit()
    
    record = cursor.fetchall()
    print("SQLite Database Version is: ", record)
    
except sqlite3.Error as error:
    print("Error while connecting to sqlite", error)
finally:
    if conn:
        conn.close()
        print("The SQLite connection is closed")
        
# Creat database connection
conn = create_engine('sqlite:///data_base_hm.sqlite',echo=False, pool_pre_ping=True)

# data insert
data_insert.to_sql( 'vitrine_one', con=conn, if_exists='append', index=False )

query = """
    SELECT * FROM vitrine_one
"""

df = pd.read_sql_query( query, conn )
df

data.to_csv('data_row.csv')

Database created and Successfully Connected to SQLite
SQLite Database Version is:  [('3.36.0',)]
The SQLite connection is closed


,product_id,style_id,color_id,product_name,color_name,product_type,Fit,product_price,messages.waistRise,shell_cotton,shell_spandex,scrapy_datetime
0,1074475001,1074475,001,men_jeans_loose,None,Loose Jeans,None,39.99,None,NaN,NaN,2022-06-12 16:35:45.000000
1,1071707001,1071707,001,men_jeans_relaxed,None,Relaxed Jeans,None,29.99,None,NaN,NaN,2022-06-12 16:35:45.000000
2,1024256001,1024256,001,men_jeans_slim,None,Slim Jeans,None,19.99,None,NaN,NaN,2022-06-12 16:35:45.000000
3,985159001,0985159,001,men_jeans_skinny,None,Skinny Jeans,None,19.99,None,NaN,NaN,2022-06-12 16:35:45.000000
4,1004199004,1004199,004,men_jeans_skinny,None,Skinny Cropped Jeans,None,29.99,None,NaN,NaN,2022-06-12 16:35:45.000000
5,1024256002,1024256,002,men_jeans_slim,None,Slim Jeans,None,19.99,None,NaN,NaN,2022-06-12 16:35:45.000000
6,690449051,0690449,051,men_jeans_ripped,None,Skinny Jeans,None,39.99,None,NaN,NaN,2022-06-12 16:35:45.000000
7,690449036,0690449,036,men_jeans_ripped,None,Skinny Jeans,None,39.99,None,NaN,NaN,2022-06-12 16:35:45.000000
8,1024256008,1024256,008,men_jeans_slim,None,Slim Jeans,None,19.99,None,NaN,NaN,2022-06-12 16:35:45.000000
9,875105018,0875105,018,men_jeans_relaxed,None,Relaxed Jeans,None,29.99,None,NaN,NaN,2022-06-12 16:35:45.000000
